### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current): # 현재일에서 바로 직전 1~3거래일을 찾음.
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)

        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:  # 현재일 대비 최대 30일전까지 거래가 있을 경우 계산 (현실적으로 최대 10일 이상 폐장되는 경우 없음)
            break
            
    return date2

In [3]:
# 거래일과 거래일 바로 이전일의 변화율을 계산함.
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
# 변화율(historical, investors), weekday를 계산하고 합하여 return함
def combine_data(df_o, df_inv, df_his):
    # df_inv : df_investor, df_his: df_historical
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    current_previous_date = {}
    
    # 최초의 0이 아닌(최초 거래가 이루어진 일자) 날짜로 부터 일주일 후부터 기준.
    # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
    for k, i in enumerate(df_inv.iloc[:, 1]):  
        if i != 0: 
            skip_num = k + 10 # 8: 전일, 9: 전전일, 10: -3일까지 확보 
            break

    for date_current in df_o.index[skip_num:]: 
        # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
        # 거래 전일(-1일) 날짜구하기

        # 거래가 없는날 (개인의 거래가 없는 날) 은 skip
        if df_investors_temp.set_index('date').loc[date_current].iloc[0] == 0: 
            continue
            
        date_previous_c = find_previous_date(df_inv, df_his, date_current)
        current_previous_date[date_current] = date_previous_c

        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous_c.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous_c)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp_1 = df_inv.loc[[date_previous_1, date_previous_c]]
        df_his_comp_1 = df_his.loc[[date_previous_1, date_previous_c]]
    
    
        # 거래 전전일(-2일) 날짜 구하기
        date_previous_2 = find_previous_date(df_inv, df_his, date_previous_1)
        # 거래 전전날(-2일째) 요일 구하기  -- 2일전 요일은 의미 없을 것 같아서 추가하지 않음.
#         date_temp_2 = {'date': date_current, 'weekday' : date_previous_2.weekday()}
#         df_temp_2 = pd.DataFrame(date_temp_2, index=[0]).set_index('date')
#         date_weekday = pd.concat([date_weekday, df_temp_2], axis=0)
        # 거래 전날(-1일), 전전날(-3일)을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp_2 = df_inv.loc[[date_previous_2, date_previous_c]]
        df_his_comp_2 = df_his.loc[[date_previous_2, date_previous_c]]
            
        # 전날 -전전날 ratio, 전날 - 전전전날 ratio, column확대
        df_inv_concat_2 = pd.concat([find_ratio(df_inv_comp_1, date_current).iloc[[-1]],
                                 find_ratio(df_inv_comp_2, date_current).iloc[[-1]]], axis=1)
        df_his_concat_2 = pd.concat([find_ratio(df_his_comp_1, date_current).iloc[[-1]],
                                 find_ratio(df_his_comp_2, date_current).iloc[[-1]]], axis=1)
        
        investor_rate = pd.concat([investor_rate,  df_inv_concat_2], axis=0)
        historical_rate = pd.concat([historical_rate, df_his_concat_2], axis=0)

    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)

    return total, current_previous_date

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}
code = {'005380' : ['현대차', 'hyunmotor']}

In [6]:
# hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol'] # close_cr 제외하고 사용. divided by zero 회피용
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [7]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 있는
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml_ex = '../data/data_for_ml/expand_date/'
directory_for_ml_ex_update = '../data/data_for_ml/expand_date/update/'
pkl_directory = '../data/company_pkl/'

for key, val in code.items():
#     pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
#     df_class_temp = pd.read_pickle(pkl_name)
#     df_class_temp['date'] = pd.to_datetime(df_class_temp['date']) #change to timestamp
#     df_class_temp['date'] = df_class_temp['date'].dt.date  # change to datetime
 
    pkl_name= '{}_historical.pkl'.format(val[1])
    df_historical_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_historical_temp = df_historical_temp[hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생), close_cr은 target column에서 재 계산하여사용
    df_historical_temp['date'] = df_historical_temp['date'].dt.date # change to datetime
    
    pkl_name= '{}_investors.pkl'.format(val[1])
    df_investors_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_investors_temp['date'] = df_investors_temp['date'].dt.date # change to datetime
    
    # ******** 시작 일자, 마지막 일자  지정 ***********
    start_date = datetime.date(2022, 1, 1) # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
    end_date = df_investors_temp['date'].iloc[-1]  # 투자자별 자료가 있는 마지막 날짜
    
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_base = pd.DataFrame(pd.Series(date_range_ts, name='date'))
    df_base['date'] = df_base['date'].dt.date
    
# get combined data (eg. df_sec_combined, df_ktng_combined...)
#     df_combined_temp = combine_data(df_class_temp.set_index('date'), df_investors_temp.set_index('date'),
#                                     df_historical_temp.set_index('date'))

    df_combined_temp, cp_date = combine_data(df_base.set_index('date'), 
                                    df_investors_temp.set_index('date'), df_historical_temp.set_index('date'))
    
    # column nama change according to the newly added columns
    column_name_change = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
          'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 'foreigneretc_1', 
          'retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2', 
          'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 'foreigneretc_2', 
          'open_1', 'high_1', 'low_1', 'close_1', 'vol_1', 
          'open_2', 'high_2', 'low_2', 'close_2', 'vol_2', 'weekday' ]
    
    df_combined_temp.columns = column_name_change
    
    df_combined_temp['temp'] = df_combined_temp['close_1'].shift(-1) # 현재날짜 증감을 확인하기 위하여 임시 컬럼 추가
    min_rate = 0.0 # +로 끝난 상황을 알기 위함
    df_combined_temp['cr_00'] = df_combined_temp['temp'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.005 # 수수료등 비용 0.2672% 이상 확인하기 위함, 0.5% 상승 마감
    df_combined_temp['cr_05'] = df_combined_temp['temp'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.010 # 1.0% 상승 마감
    df_combined_temp['cr_10'] = df_combined_temp['temp'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.015 # 1.5% 상승 마감
    df_combined_temp['cr_15'] = df_combined_temp['temp'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.020 # 2.0% 상승 마감
    df_combined_temp['cr_20'] = df_combined_temp['temp'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    
    df_combined_temp.drop(columns='temp', inplace=True) # 사용후 삭제
    
    column_selected = column_name_change
    column_selected.extend(['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20'])  # 아래 class column 이 변경에 따라 수정해야 함
    
    globals()['df_{}_{}'.format(val[1], 'combined')] = df_combined_temp.copy()
    globals()['df_{}_{}'.format(val[1], 'sel')] = df_combined_temp[column_selected]

In [8]:
df_combined_temp.iloc[:, 25:].head(10)

,high_1,low_1,close_1,vol_1,open_2,high_2,low_2,close_2,vol_2,weekday,cr_00,cr_05,cr_10,cr_15,cr_20
2022-01-12,-0.016279,0.0,0.004773,-0.355456,-0.02331,-0.023095,-0.021077,-0.02093,-0.184782,1.0,1,0,0,0,0
2022-01-13,0.007092,0.007177,0.004751,-0.017698,-0.011682,-0.009302,0.007177,0.009547,-0.366863,2.0,0,0,0,0,0
2022-01-14,0.0,-0.002375,-0.007092,0.057818,0.009547,0.007092,0.004785,-0.002375,0.039097,3.0,0,0,0,0,0
2022-01-17,-0.00939,-0.009524,-0.004762,-0.192091,-0.009456,-0.00939,-0.011876,-0.01182,-0.145379,4.0,0,0,0,0,0
2022-01-18,-0.009479,-0.026442,-0.019139,0.73891,-0.014184,-0.018779,-0.035714,-0.02381,0.404881,0.0,0,0,0,0,0
2022-01-19,-0.014354,-0.004938,-0.012195,-0.203217,-0.016706,-0.023697,-0.03125,-0.0311,0.385533,1.0,0,0,0,0,0
2022-01-20,-0.012136,-0.007444,-0.009877,-0.103097,-0.038369,-0.026316,-0.012346,-0.021951,-0.285363,2.0,1,1,0,0,0
2022-01-21,-0.004914,-0.0025,0.007481,-0.141439,-0.026699,-0.01699,-0.009926,-0.002469,-0.229954,3.0,0,0,0,0,0
2022-01-24,-0.007407,-0.005013,-0.009901,0.129685,-0.004988,-0.012285,-0.0075,-0.002494,-0.030097,4.0,0,0,0,0,0
2022-01-25,-0.004975,-0.017632,-0.015,0.263835,-0.014963,-0.012346,-0.022556,-0.024752,0.427735,0.0,0,0,0,0,0


In [9]:
'''
# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday']
'''

"\n# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.\ncolumn_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',\n              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',\n              'open', 'high', 'low', 'close', 'vol','weekday']\n"

## 주요 지표 읽기

### Read and Combine data 

In [10]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [11]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [12]:
common_col_name = {'dji.pkl':'dji', 'dji_future.pkl':'dji_f', 'dxy_future.pkl':'dxy', 
                   'ixic_future.pkl':'ixic_f', 'kor_10yr_bond.pkl':'bond_kor_10',
                   'kor_2yr_bond.pkl':'bond_kor_2', 'kosdaq.pkl':'kosdaq', 'kospi.pkl':'kospi',
                   'krw_rate.pkl':'krw', 'nas.pkl':'ixic', 'snp_future.pkl':'spx_f',
                   'sox.pkl':'sox', 'spx.pkl':'spx', 'us_10yr_bond.pkl':'bond_usa_10',
                   'us_2yr_bond.pkl':'bond_usa_2', 'us_3mon_bond.pkl':'bond_usa_3m',
                   'vix.pkl':'vix', 'wti_future.pkl':'wti' }

In [13]:
end_date = datetime.datetime.today().date()
start_date = '20220101'  # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [14]:
# pkl_common_directory = '../data/common_pkl/'
# df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# # for index_name in major_index:
# for index_name in common_pkl:
#     key_name = index_name[:-4]
# #     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
# #     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
# #                             how='left', left_index=True, right_index=True)
#     df_temp = pd.read_pickle(pkl_common_directory + index_name)
#     df_temp['date'] = df_temp['date'] + datetime.timedelta(days = 1) # 하루전 데이터와 일치시키기 위하여 하루 shift
#     df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)

In [35]:
pkl_common_directory = '../data/common_pkl/'
# df_base_t = pd.DataFrame(pd.Series(date_range_ts, name='date'))['date'].dt.date
# df_base = df_base_t.set_index('date', inplace=True)

for index_name in common_pkl:
    key_name = index_name[:-4]
    col_name = common_col_name[index_name]
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_temp['date'] = df_temp['date'].dt.date # date type으로 변환
    df_temp['temp'] = df_temp[col_name].shift(2)  # 2일전 데이터와 비교하기 위하여 shift
    df_temp[f'{col_name}_cr_2'] = (df_temp[col_name] - df_temp['temp'])/df_temp['temp']*100 # 하루전 데이터는 "_cr"
#     df_temp['date'] = df_temp['date'] + datetime.timedelta(days = 1) # 하루전 데이터와 일치시키기 위하여 하루 shift
    
#     df_temp.iloc[[1],:]
#     df_temp.iloc[:, -3] 
#     for key, value in cp_date.items():
#         if df_base.index
#     df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-3, -1]], \
#                             how='left', left_index=True, right_index=True)
    
# for date_current in df_o.index[skip_num:]: 
#     # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
#     # 거래 전일(-1일) 날짜구하기

#     # 거래가 없는날 (개인의 거래가 없는 날) 은 skip
#     if df_investors_temp.set_index('date').loc[date_current].iloc[0] == 0: 
#         continue
# df_combined_temp = combine_data(df_base.set_index('date'), 
#                                     df_investors_temp.set_index('date'), df_historical_temp.set_index('date'))
#     date_previous_c = find_previous_date(df_inv, df_his, date_current)

In [45]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [46]:
df_base

""
date
2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
...
2023-03-11
2023-03-12
2023-03-13


In [ ]:
df_base_t = pd.DataFrame(pd.Series(date_range_ts, name='date'))['date'].dt.date
df_base = df_base.set_index('date', inplace=True)

In [59]:
for i, date_t in enumerate(df_base.index):
#     print(date_t.date())
    dd =  df_temp[df_temp['date']  == date_t.date()]
    print(dd)


Empty DataFrame
Columns: [date, wti, open, high, low, volume, wti_cr, temp, wti_cr_2]
Index: []
Empty DataFrame
Columns: [date, wti, open, high, low, volume, wti_cr, temp, wti_cr_2]
Index: []
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
527  2022-01-03  76.08  75.69  76.46  74.27  317.63K  +1.16%  76.99 -1.181972
           date    wti   open   high   low   volume  wti_cr   temp  wti_cr_2
528  2022-01-04  76.99  76.02  77.64  75.7  372.85K  +1.20%  75.21  2.366707
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
529  2022-01-05  77.85  77.18  78.58  76.51  397.90K  +1.12%  76.08  2.326498
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
530  2022-01-06  79.46  77.24  80.24  76.73  476.67K  +2.07%  76.99  3.208209
           date   wti   open   high    low   volume  wti_cr   temp  wti_cr_2
531  2022-01-07  78.9  79.62  80.47  78.52  414.43K  -0.70%  77.85  1.348748
Empty DataFrame
Columns: [date, 

           date     wti   open    high    low   volume  wti_cr  temp  wti_cr_2
664  2022-07-07  102.73  98.22  104.48  96.57  379.86K  +4.26%  99.5  3.246231
           date     wti    open    high     low   volume  wti_cr   temp  \
665  2022-07-08  104.79  102.22  105.24  101.51  327.46K  +2.01%  98.53   

     wti_cr_2  
665  6.353395  
Empty DataFrame
Columns: [date, wti, open, high, low, volume, wti_cr, temp, wti_cr_2]
Index: []
Empty DataFrame
Columns: [date, wti, open, high, low, volume, wti_cr, temp, wti_cr_2]
Index: []
           date     wti    open    high     low   volume  wti_cr    temp  \
666  2022-07-11  104.09  104.79  105.05  100.89  348.57K  -0.67%  102.73   

     wti_cr_2  
666  1.323859  
           date    wti    open    high    low   volume  wti_cr    temp  \
667  2022-07-12  95.84  103.46  103.49  95.35  398.55K  -7.93%  104.79   

     wti_cr_2  
667 -8.540891  
           date   wti   open   high    low   volume  wti_cr    temp  wti_cr_2
668  2022-07-13  96.3  

           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
814  2023-01-25  80.15  80.26  81.23  79.45  297.23K  +0.02%  81.62 -1.801029
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
815  2023-01-26  81.01  80.48  82.14  79.92  291.06K  +1.07%  80.13  1.098215
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
816  2023-01-27  79.68  81.22  82.48  79.04  367.59K  -1.64%  80.15   -0.5864
Empty DataFrame
Columns: [date, wti, open, high, low, volume, wti_cr, temp, wti_cr_2]
Index: []
           date    wti   open   high    low volume  wti_cr   temp  wti_cr_2
817  2023-01-29  80.33  79.98  80.49  79.88    NaN  +0.82%  81.01 -0.839403
           date   wti   open   high    low   volume  wti_cr   temp  wti_cr_2
818  2023-01-30  77.9  80.04  80.49  77.66  327.43K  -3.03%  79.68 -2.233936
           date    wti   open   high    low   volume  wti_cr   temp  wti_cr_2
819  2023-01-31  78.87  77.91  79.25  76.55  312.73K

In [58]:
dd

,date,wti,open,high,low,volume,wti_cr,temp,wti_cr_2


In [38]:
df_temp['date'] == datetime.date(2023, 3, 8)

0      False
1      False
2      False
3      False
4      False
       ...  
848    False
849    False
850     True
851    False
852    False
Name: date, Length: 853, dtype: bool

In [23]:
df_temp.tail()

,date,wti,open,high,low,volume,wti_cr,temp,wti_cr_2
848,2023-03-06,80.46,79.92,80.63,78.32,292.35K,+1.34%,79.68,0.978916
849,2023-03-07,77.58,80.50,80.94,77.06,398.24K,-3.58%,79.40,-2.292191
850,2023-03-08,76.66,77.32,77.73,76.11,335.93K,-1.19%,80.46,-4.722844
851,2023-03-09,75.45,76.56,78.05,75.36,NaN,-1.58%,77.58,-2.745553
852,2023-03-10,75.59,75.45,75.61,75.42,NaN,+0.19%,76.66,-1.395774


In [24]:
datetime.date(2023, 3, 8)

datetime.date(2023, 3, 8)

In [37]:
df_temp['date'] = df_temp['date'].dt.date

In [32]:
df_temp

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-06
4      2020-01-07
          ...    
848    2023-03-06
849    2023-03-07
850    2023-03-08
851    2023-03-09
852    2023-03-10
Name: date, Length: 853, dtype: object

In [27]:
cp_date

{datetime.date(2022, 1, 12): datetime.date(2022, 1, 11),
 datetime.date(2022, 1, 13): datetime.date(2022, 1, 12),
 datetime.date(2022, 1, 14): datetime.date(2022, 1, 13),
 datetime.date(2022, 1, 17): datetime.date(2022, 1, 14),
 datetime.date(2022, 1, 18): datetime.date(2022, 1, 17),
 datetime.date(2022, 1, 19): datetime.date(2022, 1, 18),
 datetime.date(2022, 1, 20): datetime.date(2022, 1, 19),
 datetime.date(2022, 1, 21): datetime.date(2022, 1, 20),
 datetime.date(2022, 1, 24): datetime.date(2022, 1, 21),
 datetime.date(2022, 1, 25): datetime.date(2022, 1, 24),
 datetime.date(2022, 1, 26): datetime.date(2022, 1, 25),
 datetime.date(2022, 1, 27): datetime.date(2022, 1, 26),
 datetime.date(2022, 1, 28): datetime.date(2022, 1, 27),
 datetime.date(2022, 2, 3): datetime.date(2022, 1, 28),
 datetime.date(2022, 2, 4): datetime.date(2022, 2, 3),
 datetime.date(2022, 2, 7): datetime.date(2022, 2, 4),
 datetime.date(2022, 2, 8): datetime.date(2022, 2, 7),
 datetime.date(2022, 2, 9): datetime.d

In [ ]:
def find_loc(df1, df2, val):
    df1[df1['date'] == val]]

In [ ]:
df_temp.set_index('date', inplace=True)

In [ ]:
df_temp.iloc[[1],:]

In [ ]:
df_base.head()

In [ ]:
df_base.loc[datetime.datetime(2022, 1, 4), :]

In [ ]:
'''
df_sec_sel = df_sec_combined[column_selected]
'''

In [ ]:
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df_base, how='left', left_index=True, right_index=True)

In [ ]:
df_hyunmotor_sel.iloc[:, 10:20].head(20)

In [ ]:
'''
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)
'''

In [ ]:
# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
# weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
new_columns = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
               'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 'foreigneretc_1', 
               'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr', 'bond_kor_10_cr',
               'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
               'kosdaq_cr', 'kospi_cr', 'krw_cr', 'sox_cr', 'vix_cr', 'wti_cr',                
               'open_1', 'high_1', 'low_1', 'close_1', 'vol_1',      
               'retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
               'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 'foreigneretc_2',
               'dji_cr_2', 'dji_f_cr_2', 'ixic_cr_2', 'ixic_f_cr_2', 'spx_cr_2', 'spx_f_cr_2',
               'bond_kor_10_cr_2', 'bond_kor_2_cr_2', 'dxy_cr_2', 'bond_usa_10_cr_2','bond_usa_2_cr_2', 
               'bond_usa_3m_cr_2', 'kosdaq_cr_2', 'kospi_cr_2', 'krw_cr_2', 'sox_cr_2', 'vix_cr_2', 'wti_cr_2', 
#                 'open', 'high', 'low', 'close', 'vol', 'weekday',  # close column 제거함 (class column 계산항목으로 사용됨)
                'open_2', 'high_2', 'low_2', 'close_2', 'vol_2', 'weekday', 
                'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [ ]:
# column 순서 변경 : weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = globals()['df_{}_sel'.format(val[1])][new_columns]

In [ ]:
df_hyunmotor_sel.iloc[:, 10:20].head(20)

In [ ]:
'''
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)

.impute 사용하는 것을 고려할 필요 있음.
'''

In [ ]:
# # drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# # 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
# for key, val in code.items():
#     globals()['df_{}_sel'.format(val[1])].replace([np.inf, -np.inf], [1, -1], inplace=True)

# **** 데이터 전처리 from sklearn.impute import SimpleImputer, SimpleImputer 사용하기로 함.

In [ ]:
# # delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
# df_sec_sel.dropna(inplace=True)

In [ ]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].dropna(inplace=True)

In [ ]:
df_hyunmotor_sel.head()

In [ ]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)
'''
df_sec_sel = string_to_num(df_sec_sel)
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = string_to_num(globals()['df_{}_sel'.format(val[1])])

In [ ]:
# pickle로 데이터 저장
'''
directory_for_ml_ex_update = '../data/data_for_ml/expand_date/update/'
df_sec_sel.to_pickle(directory_for_ml_ex_update + 'df_sec_sel.pkl')
df_sec_sel.to_csv(directory_for_ml_ex_update + 'df_sec_sel.csv')
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].to_pickle(directory_for_ml_ex + 'df_{}_sel.pkl'.format(val[1]))
    globals()['df_{}_sel'.format(val[1])].to_csv(directory_for_ml_ex + 'df_{}_sel.csv'.format(val[1]))